In [1]:
from fastai.text import * 
from fastai.callbacks import *
from sklearn.model_selection import train_test_split
from pathlib import Path
import pickle as pkl

torch.cuda.set_device(1)

In [2]:
import unicodedata
import pandas as pd
import re
import spacy
import json

In [3]:
PATH = Path('.')

In [4]:
def normalize_title(title):
    s = re.sub(r'[^a-zA-Z0-9ñç% ]', ' ', unicodedata.normalize('NFKD', title.lower()).encode('ascii', 'ignore').decode("utf-8"))
#    s = re.sub(r'[\d]+', "1", s)
    s = re.sub(r's |s$', ' ', s)
    s = re.sub(r' +', ' ', s)
    s = re.sub(r'(1 )+', '1 ', s)
    return re.sub(r'o |o$', 'a ', s).strip()

In [5]:
with open("main_categories.json", "r") as f:
    cats = json.load(f)

In [6]:
vocab = Vocab.load('models/v4_vocab_por' + '.pkl')

In [7]:
df = pd.read_csv(PATH / 'train.csv')
df = df[df.language == 'portuguese']
df.title = df.title.apply(normalize_title)
df = df[~df.title.isna() & (df.title != 'nan') & (df.title != '')]

/home/jdg/miniconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df['main_category'] = df.category.map(cats)

In [12]:
processor = SPProcessor.load('',tmp_dir='lm_por_v4')

for cat in set(cats.values()):
    d = df[df.main_category == cat]
    print(cat, len(d))
    if len(d):
        train, valid = train_test_split(d, test_size=0.1, stratify=d.category, random_state = 42)
        data_class = ItemLists('', TextList.from_df(train, vocab=vocab, cols=['title'], processor=processor),
                                TextList.from_df(valid, vocab=vocab, cols=['title'], processor=processor))
        data_class = data_class.label_from_df(cols=['category'])
        data_class = data_class.databunch(bs=800, num_workers=1)      
        data_class.save(f'/data/anime/ml/por/{cat}.pkl')

In [13]:
from sklearn.metrics import f1_score

@np_func
def f1(inp,targ): 
    y_pred = np.argmax(inp, axis=-1)
    return f1_score(targ, y_pred, average='weighted',  labels=np.unique(y_pred))

class WeightedLabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, weight, eps:float=0.1, reduction='mean'):
        super().__init__()
        self.weight,self.eps,self.reduction = weight,eps,reduction
        
    def forward(self, output, target):
        c = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        if self.reduction=='sum': loss = -log_preds.sum()
        else:
            loss = -log_preds.sum(dim=-1)
            if self.reduction=='mean':  loss = loss.mean()
        return loss*self.eps/c + (1-self.eps) * F.nll_loss(log_preds, target, weight=self.weight, reduction=self.reduction)

In [18]:
for cat in set(cats.values()):
    print(cat)
    if not (Path(f'/data/anime/ml/por/{cat}_model.pth')).exists():
        data_class = load_data('/data/anime/ml/por/', cat + '.pkl', bs=500)
        num_trn = len(data_class.train_ds.x)
        num_val = len(data_class.valid_ds.x)
        num_trn, num_val, num_trn+num_val
        trn_LabelCounts = np.unique(data_class.train_ds.y.items, return_counts=True)[1]
        val_LabelCounts = np.unique(data_class.valid_ds.y.items, return_counts=True)[1]
        trn_LabelCounts, val_LabelCounts
        trn_weights = [1 - count/num_trn for count in trn_LabelCounts]
        val_weights = [1 - count/num_val for count in val_LabelCounts]
        trn_weights, val_weights
        
        config = awd_lstm_clas_config.copy()
        config['qrnn'] = True
        config['n_hid'] = 1550 #default 1152
        config['n_layers'] = 4 #default 3      
        
        learn_c = text_classifier_learner(data_class, AWD_LSTM, pretrained=False, config=config, drop_mult=0.3)
        learn_c.load_encoder('best_lm_por_v4_encoder');
        
        loss_weights = torch.FloatTensor(trn_weights).cuda()
        learn_c.loss_func = FlattenedLoss(WeightedLabelSmoothingCrossEntropy, weight=loss_weights)
        
        learn_c.freeze()
        learn_c.fit_one_cycle(2, 3e-2, moms=(0.8,0.7))
        
        learn_c.freeze_to(-2)
        learn_c.fit_one_cycle(2, 3e-2, wd=0.01, moms=(0.8,0.7))
        
        lr = 3e-2
        wd = 0.01
        learn_c.freeze_to(-3)
        learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), wd=wd, moms=(0.8,0.7))
        
        learn_c.unfreeze()
        learn_c.fit_one_cycle(2, slice(lr/10/(2.6**4),lr/10), wd=wd, moms=(0.8,0.7))
        
        learn_c.save(f'/data/anime/ml/por/{cat}_model')
        del learn_c
        torch.cuda.empty_cache()
        gc.collect()

Musica, Filmes e Seriados


epoch,train_loss,valid_loss,accuracy,time
0,0.597866,0.571005,0.878863,00:05
1,0.553747,0.490856,0.900798,00:05


epoch,train_loss,valid_loss,accuracy,time
0,0.514318,0.472029,0.918744,00:05
1,0.484327,0.455096,0.924227,00:05


epoch,train_loss,valid_loss,accuracy,time
0,0.452514,0.493933,0.923230,00:06
1,0.437799,0.445883,0.929711,00:06


epoch,train_loss,valid_loss,accuracy,time
0,0.408890,0.443828,0.931206,00:08
1,0.403202,0.446752,0.930209,00:08


...

Livros, Revistas e Comics


epoch,train_loss,valid_loss,accuracy,time
0,0.853895,0.764218,0.763573,00:07
1,0.799639,0.741912,0.778459,00:07


epoch,train_loss,valid_loss,accuracy,time
0,0.775281,0.747197,0.772329,00:07
1,0.715195,0.690426,0.806480,00:07


epoch,train_loss,valid_loss,accuracy,time
0,0.694489,0.693555,0.809107,00:08
1,0.647932,0.680557,0.812026,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.607370,0.683403,0.814653,00:12
1,0.601936,0.689030,0.815528,00:12


In [23]:
cat = 'Livros, Revistas e Comics'
learn_c = text_classifier_learner(data_class, AWD_LSTM, pretrained=False, config=config, drop_mult=0.5)
learn_c.load(f'/data/anime/ml/por/{cat}_model')
learn_c.fit_one_cycle(3, slice(lr/10/(2.6**4),lr/10), wd=wd, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.464418,0.453853,0.815236,00:12
1,0.447053,0.457287,0.810274,00:12
2,0.434355,0.455159,0.810274,00:12


In [25]:
learn_c.show_results(rows=300)

text,target,prediction
▁xxbos ▁440 7 ▁manga ▁san ct ua ry ▁1 ▁sha ▁f omi m ura ▁conrad ▁4 ▁ ry oi chi ▁ ike g ami,MANGA,MANGA
▁xxbos ▁livra ▁pro g re ▁de ▁la ▁m eta phy s ique ▁en ▁all ema g ne ▁de pu i ▁lei b ni,EROTIC_BOOKS,EROTIC_BOOKS
▁xxbos ▁livra ▁ oe u v re ▁t ▁vi ▁rap port ▁du ▁ phy s ique ▁et ▁du ▁mor al ▁de ▁l,EROTIC_BOOKS,EROTIC_BOOKS
▁xxbos ▁livra ▁voce ▁de c ide ▁com a ▁se ▁s ente ▁ga ry ▁mc ka y ▁e ▁don ▁din k me yer,BOOKS,BOOKS
▁xxbos ▁metoda ▁piana ▁bela ▁bar to k ▁mi k rok os ma ▁vol ▁5 ▁p ▁r ▁a ▁m ▁a ▁c ▁a ▁a,BOOKS,BOOKS
▁xxbos ▁caca da ▁aa ▁poderosa ▁chef aa ▁spa w n ▁a ▁mal dica a ▁da ▁spa w n ▁24 h q,COMICS,BOOKS
▁xxbos ▁livra ▁p our ▁un e ▁union ▁oc ci d enta le ▁entre ▁l ▁euro pe ▁et ▁le ▁ eta t,EROTIC_BOOKS,EROTIC_BOOKS
▁xxbos ▁re ▁integra tion ▁of ▁the ▁ schi zo ph ren ic ▁pa ti ent ▁j ▁a ▁ lie ber man,BOOKS,BOOKS
▁xxbos ▁revista ▁cont iga ▁2 103 ▁van essa ▁ gia com a ▁an itta ▁we s ley ▁sa f ada a,BOOKS,BOOKS
▁xxbos ▁mo tive ▁a 2 ▁ar be it sb uch ▁le k tion ▁9 ▁18 ▁mi t ▁mp 3 ▁audia ▁c,EROTIC_BOOKS,EROTIC_BOOKS


In [7]:
for cat in set(cats.values()):
    print(cat)
    config = awd_lstm_clas_config.copy()
    config['qrnn'] = True
    config['n_hid'] = 1550 #default 1152
    config['n_layers'] = 4 #default 3 
    data_class = load_data('/data/anime/ml/por/', cat + '.pkl', bs=500)
    learn_c = text_classifier_learner(data_class, AWD_LSTM, pretrained=False, config=config, drop_mult=0.3)
    learn_c.load(f'/data/anime/ml/por/{cat}_model')
    learn_c.export(f'/data/anime/ml/por/{cat}_model_export')
    del learn_c
    torch.cuda.empty_cache()
    gc.collect()

Musica, Filmes e Seriados
Casa, Moveis e Decoracao
Eletrodomesticos
Livros, Revistas e Comics
Antiguidades e Colecoes
Agro, Industria e Comercio
Bebes
Games
Cameras e Acessorios
Ingressos
Joias e Relogios
Informatica
Carros, Motos e Outros
Brinquedos e Hobbies
Acessorios para Veiculos
Celulares e Telefones
Esportes e Fitness
Ferramentas e Construcao
Eletronicos, Audio e Video
Saude
Alimentos e Bebidas
Calcados, Roupas e Bolsas
Beleza e Cuidado Pessoal
Instrumentos Musicais
Festas e Lembrancinhas
Mais Categorias
Animais
Arte, Papelaria e Armarinho
